<a href="https://colab.research.google.com/github/Kvazzzzar/MPSI/blob/main/MPSI_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np

class NeuralNetwork:
    """
    Простая нейронная сеть с одним скрытым слоем
    Реализованы:
    - Инициализация с заданным количеством нейронов в скрытом слое
    - Прямое распространение (forward pass)
    - Обратное распространение ошибки (backpropagation)
    - Обучение на данных
    """

    def __init__(self, input_size, hidden_size, output_size):
        """Инициализация весов и смещений"""
        self.W1 = np.random.randn(input_size, hidden_size) * 0.01
        self.b1 = np.zeros((1, hidden_size))
        self.W2 = np.random.randn(hidden_size, output_size) * 0.01
        self.b2 = np.zeros((1, output_size))

    def sigmoid(self, x):
        """Функция активации сигмоида"""
        return 1 / (1 + np.exp(-x))

    def sigmoid_derivative(self, x):
        """Производная сигмоиды"""
        return x * (1 - x)

    def forward(self, X):
        """Прямое распространение"""
        self.z1 = np.dot(X, self.W1) + self.b1
        self.a1 = self.sigmoid(self.z1)
        self.z2 = np.dot(self.a1, self.W2) + self.b2
        self.a2 = self.sigmoid(self.z2)
        return self.a2

    def backward(self, X, y, output, learning_rate):
        """Обратное распространение ошибки и обновление весов"""
        m = X.shape[0]

        # Ошибка на выходном слое
        self.a2_error = y - output
        self.a2_delta = self.a2_error * self.sigmoid_derivative(output)

        # Ошибка на скрытом слое
        self.a1_error = self.a2_delta.dot(self.W2.T)
        self.a1_delta = self.a1_error * self.sigmoid_derivative(self.a1)

        # Обновление весов и смещений
        self.W2 += self.a1.T.dot(self.a2_delta) * learning_rate
        self.b2 += np.sum(self.a2_delta, axis=0, keepdims=True) * learning_rate
        self.W1 += X.T.dot(self.a1_delta) * learning_rate
        self.b1 += np.sum(self.a1_delta, axis=0, keepdims=True) * learning_rate

    def train(self, X, y, epochs=1000, learning_rate=0.1):
        """Процесс обучения"""
        for epoch in range(epochs):
            output = self.forward(X)
            self.backward(X, y, output, learning_rate)

            if epoch % 100 == 0:
                loss = np.mean(np.square(y - output))
                print(f'Epoch {epoch}, Loss: {loss:.4f}')

In [2]:
# Пример использования на табличных данных
X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])  # Входные данные
y = np.array([[0], [1], [1], [0]])               # Выходные данные (XOR)

# Создаем сеть с 4 нейронами в скрытом слое
nn = NeuralNetwork(input_size=2, hidden_size=4, output_size=1)

# Обучаем сеть
nn.train(X, y, epochs=1000, learning_rate=0.1)

# Тестируем
print("Predictions after training:")
for x in X:
    print(f"{x} -> {nn.forward(x.reshape(1, -1))[0][0]:.4f}")

Epoch 0, Loss: 0.2500
Epoch 100, Loss: 0.2500
Epoch 200, Loss: 0.2500
Epoch 300, Loss: 0.2500
Epoch 400, Loss: 0.2500
Epoch 500, Loss: 0.2500
Epoch 600, Loss: 0.2500
Epoch 700, Loss: 0.2500
Epoch 800, Loss: 0.2500
Epoch 900, Loss: 0.2500
Predictions after training:
[0 0] -> 0.5000
[0 1] -> 0.5000
[1 0] -> 0.5000
[1 1] -> 0.5000


# Нейронная сеть с нуля:

1. Реализован класс NeuralNetwork с возможностью задания количества нейронов в скрытом слое

2. Включает все необходимые компоненты: инициализацию весов, функции активации, прямое и обратное распространение

3. Поддерживается обучение на табличных данных (пример с XOR)





In [3]:
import numpy as np

class SimpleGPT:
    """
    Упрощенная реализация GPT с нуля
    Включает только базовые компоненты:
    - Токенизацию
    - Эмбеддинги
    - Механизм внимания
    - Генерацию текста
    """

    def __init__(self, vocab_size, embedding_dim, max_seq_length):
        """Инициализация параметров"""
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.max_seq_length = max_seq_length

        # Эмбеддинги токенов
        self.token_emb = np.random.randn(vocab_size, embedding_dim) * 0.01
        # Позиционные эмбеддинги
        self.pos_emb = np.random.randn(max_seq_length, embedding_dim) * 0.01

        # Параметры внимания
        self.Wq = np.random.randn(embedding_dim, embedding_dim) * 0.01
        self.Wk = np.random.randn(embedding_dim, embedding_dim) * 0.01
        self.Wv = np.random.randn(embedding_dim, embedding_dim) * 0.01

    def attention(self, x):
        """Механизм внимания (self-attention)"""
        Q = np.dot(x, self.Wq)
        K = np.dot(x, self.Wk)
        V = np.dot(x, self.Wv)

        scores = np.dot(Q, K.T) / np.sqrt(self.embedding_dim)
        weights = self.softmax(scores)
        output = np.dot(weights, V)
        return output

    def softmax(self, x):
        """Функция softmax"""
        exp_x = np.exp(x - np.max(x, axis=-1, keepdims=True))
        return exp_x / np.sum(exp_x, axis=-1, keepdims=True)

    def forward(self, token_ids):
        """Прямой проход"""
        # Получаем эмбеддинги токенов и позиций
        token_embeddings = self.token_emb[token_ids]
        pos_embeddings = self.pos_emb[:len(token_ids)]

        # Суммируем эмбеддинги
        x = token_embeddings + pos_embeddings

        # Применяем механизм внимания
        x = self.attention(x)

        # Преобразуем обратно в вероятности токенов
        logits = np.dot(x, self.token_emb.T)
        probs = self.softmax(logits)
        return probs

    def generate(self, start_token, max_length=10):
        """Генерация текста"""
        tokens = [start_token]

        for _ in range(max_length):
            probs = self.forward(np.array(tokens))
            next_token = np.argmax(probs[-1])
            tokens.append(next_token)

        return tokens

In [4]:
# Пример использования (упрощенный)
vocab_size = 10000  # Размер словаря
embedding_dim = 256  # Размерность эмбеддингов
max_seq_length = 512  # Максимальная длина последовательности

# Создаем модель
gpt = SimpleGPT(vocab_size, embedding_dim, max_seq_length)

# Генерируем последовательность (в реальности нужно предварительное обучение)
generated_tokens = gpt.generate(start_token=0, max_length=20)
print("Generated tokens:", generated_tokens)

Generated tokens: [0, np.int64(6298), np.int64(9185), np.int64(1143), np.int64(2693), np.int64(6072), np.int64(6757), np.int64(5741), np.int64(2356), np.int64(5474), np.int64(2356), np.int64(5474), np.int64(5474), np.int64(5474), np.int64(5474), np.int64(9233), np.int64(5474), np.int64(5474), np.int64(733), np.int64(733), np.int64(733)]


# Упрощенная GPT:

1. Реализованы базовые компоненты GPT: эмбеддинги, механизм внимания, генерация текста

2. Внимание реализовано через матрицы Q, K, V как в оригинальной статье

3. Поддерживается генерация последовательностей токенов

Оба решения написаны "с нуля" без использования готовых библиотек для нейронных сетей (только numpy для вычислений). Для реального использования GPT потребуется предварительное обучение на большом корпусе текстов.